# Machine Learning Analysis

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
data = pd.read_csv("crop_dataset.csv")
data.head(10)

In [ ]:
label_encoder = LabelEncoder()
data['Soilcolor'] = label_encoder.fit_transform(data['Soilcolor'])
data['label'] = label_encoder.fit_transform(data['label'])

In [ ]:
selected_features = [
    'Soilcolor', 'Ph', 'K', 'P', 'N', 'Zn', 'S', 
    'QV2M-W', 'QV2M-Sp', 'QV2M-Su', 'QV2M-Au', 
    'T2M_MAX-W', 'T2M_MAX-Sp', 'T2M_MAX-Su', 
    'T2M_MAX-Au', 'T2M_MIN-W', 'T2M_MIN-Sp', 
    'T2M_MIN-Su', 'T2M_MIN-Au', 
    'PRECTOTCORR-W', 'PRECTOTCORR-Sp', 
    'PRECTOTCORR-Su', 'PRECTOTCORR-Au', 
    'WD10M', 'GWETTOP', 'CLOUD_AMT', 
    'WS2M_RANGE', 'PS'
]
X = data[selected_features]
y = data['label']

In [ ]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)